# Tutorial: OpenClaw NadFun Prompt Spec (Strict vFinal)

목표: 펀드 채팅방 사용자들이 전략 Intent를 식별하고,
검증(Validity) + 가치판단(Judgment Majority) 합의 후 Relayer가 settlement 하도록 플로우를 고정한다.


## 1) Roles & Scope

- Relayer Next Server: 수집/집계/온체인 제출
- Strategy MoltBot: snapshot 기반 Intent 제안
- Participant MoltBot: 마이닝/검증/가치판단 투표(개별 커스텀 정책)

공통 스코프 키: `fundId`, `roomId`, `epochId`


In [ ]:
SCOPE = {"fundId": "fund-001", "roomId": "telegram-room-abc", "epochId": 12}
REQUIRED_HASHES = ["claimHash", "snapshotHash", "intentHash"]
SCOPE, REQUIRED_HASHES


## 2) Mining Data Spec

### Off-chain
- X feed 수집
- 채팅방 대화 수집

### On-chain
- NadFun 제공 데이터 수집(커브/이벤트/상태)


In [ ]:
OFFCHAIN_MINING_SPEC = {
    "x_feed": {"fields": ["postId", "author", "timestamp", "textHash", "engagement"]},
    "chat_room": {"fields": ["messageId", "userId", "timestamp", "textHash", "reactionStats"]},
}
ONCHAIN_MINING_SPEC = {
    "nadfun": {"network": [10143, 143], "fields": ["token", "curveState", "tradeEvents", "graduationState"]}
}
OFFCHAIN_MINING_SPEC, ONCHAIN_MINING_SPEC


## 3) Claim / Observation Schema


In [ ]:
CLAIM_PAYLOAD_V1 = {
    "schemaId": "NADFUN_SIGNAL_V1",
    "fundId": "fund-001",
    "roomId": "telegram-room-abc",
    "epochId": 12,
    "sourceType": "OFFCHAIN_X|OFFCHAIN_CHAT|ONCHAIN_NADFUN",
    "sourceRef": "...",
    "extractor": {"type": "JSONPath|Regex|ABI", "rule": "..."},
    "extracted": "...",
    "extractedType": "string|uint|float|json",
    "timestamp": 1739000000,
    "responseHash": "0x...",
    "evidenceURI": "ipfs://...",
    "miner": "0xParticipant",
}
CLAIM_HASH_ORDER = ["schemaId","fundId","roomId","epochId","sourceType","sourceRef","extractor","extracted","extractedType","timestamp","responseHash","evidenceURI","miner"]
CLAIM_PAYLOAD_V1, CLAIM_HASH_ORDER


## 4) Claim Validity Attestation


In [ ]:
CLAIM_VALIDITY_ATTEST = {
    "fundId": "fund-001",
    "roomId": "telegram-room-abc",
    "epochId": 12,
    "subjectType": "CLAIM",
    "subjectHash": "0xClaimHash",
    "verdict": "PASS|FAIL",
    "verifier": "0xVerifier",
    "nonce": 1,
    "expiresAt": 1739000900,
    "sig": "0x...",
}
CLAIM_VALIDITY_ATTEST


## 5) Snapshot Finalization


In [ ]:
SNAPSHOT_V0 = {
    "fundId": "fund-001",
    "epochId": 12,
    "orderedClaimHashes": ["0xaaa", "0xbbb", "0xccc"],
    "snapshotHash": "0x...",
    "claimCount": 3,
    "timestamp": 1739001000,
}
SNAPSHOT_V0


## 6) Intent Schema

Strategy는 finalized snapshot 기반으로 Intent만 제안한다.


In [ ]:
INTENT_V1 = {
    "intentVersion": "V1",
    "fundId": "fund-001",
    "roomId": "telegram-room-abc",
    "epochId": 12,
    "vault": "0xVault",
    "action": "BUY|SELL|HOLD",
    "tokenIn": "0xUSDC",
    "tokenOut": "0xTOKEN",
    "amountIn": "500",
    "minAmountOut": "123",
    "deadline": 1739003600,
    "maxSlippageBps": 70,
    "snapshotHash": "0x...",
    "reason": "...",
}
INTENT_HASH_ORDER = ["intentVersion","fundId","epochId","vault","action","tokenIn","tokenOut","amountIn","minAmountOut","deadline","maxSlippageBps","snapshotHash"]
INTENT_V1, INTENT_HASH_ORDER


## 7) Intent Attestation: 2-Stage

### Stage A: Validity (기술 검증)
### Stage B: Judgment (사용자 커스텀 가치판단)


In [ ]:
INTENT_VALIDITY_ATTEST = {
    "fundId": "fund-001",
    "roomId": "telegram-room-abc",
    "epochId": 12,
    "subjectType": "INTENT_VALIDITY",
    "intentHash": "0xIntentHash",
    "verdict": "PASS|FAIL",
    "verifier": "0xVerifier",
    "nonce": 7,
    "expiresAt": 1739001900,
    "sig": "0x...",
}

INTENT_JUDGMENT_VOTE = {
    "fundId": "fund-001",
    "roomId": "telegram-room-abc",
    "epochId": 12,
    "subjectType": "INTENT_JUDGMENT",
    "intentHash": "0xIntentHash",
    "vote": "YES|NO|ABSTAIN",
    "reason": "...",
    "customPolicyRef": "user-local-policy://my-room-alpha-v2",
    "voter": "0xParticipant",
    "nonce": 101,
    "expiresAt": 1739002100,
    "sig": "0x...",
}
INTENT_VALIDITY_ATTEST, INTENT_JUDGMENT_VOTE


## 8) EIP-712 Typed Data


In [ ]:
EIP712_DOMAIN = {"name": "OpenClaw", "version": "1", "chainId": 10143, "verifyingContract": "0xIntentBookOrClaimBook"}
CLAIM_ATTEST_TYPE = {"ClaimAttestation": [{"name": "fundId", "type": "string"},{"name": "epochId", "type": "uint64"},{"name": "claimHash", "type": "bytes32"},{"name": "verdict", "type": "uint8"},{"name": "nonce", "type": "uint256"},{"name": "expiresAt", "type": "uint64"}]}
INTENT_VALIDITY_TYPE = {"IntentValidity": [{"name": "fundId", "type": "string"},{"name": "epochId", "type": "uint64"},{"name": "intentHash", "type": "bytes32"},{"name": "verdict", "type": "uint8"},{"name": "nonce", "type": "uint256"},{"name": "expiresAt", "type": "uint64"}]}
INTENT_JUDGMENT_TYPE = {"IntentJudgment": [{"name": "fundId", "type": "string"},{"name": "epochId", "type": "uint64"},{"name": "intentHash", "type": "bytes32"},{"name": "vote", "type": "uint8"},{"name": "nonce", "type": "uint256"},{"name": "expiresAt", "type": "uint64"}]}
EIP712_DOMAIN, CLAIM_ATTEST_TYPE, INTENT_VALIDITY_TYPE, INTENT_JUDGMENT_TYPE


## 9) Relayer Aggregation & Settlement Gate


In [ ]:
POLICY = {
    "intentValidityThreshold": 5,
    "judgmentMinParticipation": 7,
    "judgmentMinYesRatio": 0.60,
}

def settlement_gate(validity_pass_count, yes, no, abstain, policy=POLICY):
    validity_ok = validity_pass_count >= policy["intentValidityThreshold"]
    participation = yes + no + abstain
    participation_ok = participation >= policy["judgmentMinParticipation"]
    yes_ratio = (yes / (yes + no)) if (yes + no) > 0 else 0.0
    majority_ok = yes_ratio >= policy["judgmentMinYesRatio"]
    return {
        "validity_ok": validity_ok,
        "participation_ok": participation_ok,
        "majority_ok": majority_ok,
        "can_settle": validity_ok and participation_ok and majority_ok,
        "yes_ratio": yes_ratio,
    }

settlement_gate(validity_pass_count=6, yes=8, no=3, abstain=1)


## 10) End-to-End Flow

1. Participant 마이닝(Off-chain + On-chain)
2. Claim validity attest
3. Snapshot finalize
4. Strategy intent propose
5. Intent validity attest
6. Intent judgment majority vote (각 유저 커스텀 정책)
7. Relayer 집계 후 gate 통과 시 settlement


In [ ]:
FLOW_CHECKPOINTS = [
    "claim_validated",
    "snapshot_finalized",
    "intent_proposed",
    "intent_validity_threshold_met",
    "intent_judgment_majority_met",
    "onchain_settlement_submitted",
]
FLOW_CHECKPOINTS


## 11) Prompt Assembly


In [ ]:
from pathlib import Path
PROMPT_DIR = Path.cwd() / 'prompts' / 'kr'
PROMPT_FILES = ['base_system.md','strategy_moltbot.md','participant_moltbot.md','relayer_next_server.md']
prompts = {f: (PROMPT_DIR / f).read_text(encoding='utf-8') for f in PROMPT_FILES}
{f: len(prompts[f]) for f in PROMPT_FILES}


## 12) Ops Checklist

- [ ] snapshotHash 없는 intent 차단
- [ ] duplicate/expired/unauthorized 서명 제거
- [ ] validity threshold + judgment majority 동시 충족 확인
- [ ] 사용자 customPolicyRef를 judgment 메타로 저장
- [ ] tx hash 없는 settlement 완료 알림 금지
